In [4]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import  ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
class Tagging(BaseModel):
    """Tags the pieces of text with particular Information"""
    sentiment: str = Field(description="sentiment text should be `pos`, `neg` or `neutral` ")
    language: str = Field(description="language of text(should be ISO 639-1 code)")

In [9]:
tagging_function = convert_pydantic_to_openai_function(Tagging)

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think Carefully & then tag the text as instructed"),
    ("user", "{input}")
])

model = ChatOpenAI( model="gpt-3.5-turbo" ,temperature=0)

model_with_function = model.bind(functions=[tagging_function],
                                function_call={"name": "Tagging"})

tagging_chain = prompt | model_with_function

In [12]:
tagging_chain.invoke({"input": "I Like Langchain?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 107, 'total_tokens': 117, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-060e5312-48ec-407f-bec6-4ea98490f311-0', usage_metadata={'input_tokens': 107, 'output_tokens': 10, 'total_tokens': 117, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [13]:
tagging_chain.invoke({"input": "I Like Langchain? translate in turkish"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"tr"}', 'name': 'Tagging'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5fed425a-0521-4273-8514-fd2120b742f0-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = prompt | model_with_function | JsonOutputFunctionsParser()

In [15]:
tagging_chain.invoke({"input": "I Like Langchain?"})

{'sentiment': 'pos', 'language': 'en'}